In [2]:
import requests

print(requests.get("http://localhost:8000/hello", params={"name": "Theodore"}).json())


Hello Theodore!


In [3]:
import requests

def download_image(image_url, save_path):
    try:
        response = requests.get(image_url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            print(f"Image successfully downloaded: {save_path}")
        else:
            print("Failed to download image. HTTP Status Code:", response.status_code)
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
image_url = "https://unsplash.com/photos/black-and-white-cat-lying-on-brown-bamboo-chair-inside-room-gKXKBY-C-Dk"  # Replace with the actual URL
save_path = "downloaded_image.jpg"  # Replace with your desired file path
download_image(image_url, save_path)


Image successfully downloaded: downloaded_image.jpg
